# Coursework 1 - Banking Regulation
1. (50%) The file “Dataset.xlsx” shows the total bond portfolio of a small bank in Ontario and the yield curves for the Canadian Sovereigns for the last 20 years. We will focus in this exercise only on the corporate and sovereign bond market. The bank has been categorized by the OSFI so that it must keep a capital adequacy requirement of 11.5% of its Risk-Weighted Assets. With this information, and assuming the bank does not have any other operations:

### 1(a)(20%) Using the **yield curve at the purchasing date** of the bond, calculate the **spread** the bank used to obtain the bond price.

In [1]:
# Import the data
import pandas as pd
import numpy as np
bank_loans = pd.read_excel('Dataset2020.xlsx',sheet_name = 0)
yield_curves = pd.read_excel('Dataset2020.xlsx',sheet_name = 1)

In [ ]:
# Check types of variables
bank_loans.dtypes

In [3]:
# Check types of variables
yield_curves.dtypes

Nr                  int64
Date       datetime64[ns]
 025YR            float64
 050YR            float64
 075YR            float64
                ...      
 2900YR           float64
 2925YR           float64
 2950YR           float64
 2975YR           float64
 3000YR           float64
Length: 122, dtype: object

In [4]:
# For tests only
#bank_loans = bank_loans.head(15)

In [4]:
# New column: PurchasePrice
bank_loans['PriceAtCoupon0(%)'] = bank_loans['PriceAtCoupon0(%)']/100
bank_loans['PurchasePrice'] = bank_loans['PriceAtCoupon0(%)'].mul(bank_loans['FaceValue'])

In [5]:
# Period Factor:1,4,12
bank_loans['PeriodFactor'] = 1
bank_loans['PeriodFactor'] = np.where(bank_loans['PaymentPeriod'] == 'Monthly', 12, bank_loans['PeriodFactor'])
bank_loans['PeriodFactor'] = np.where(bank_loans['PaymentPeriod'] == 'Quarterly', 4, bank_loans['PeriodFactor'])

In [6]:
bank_loans['PeriodFactor'].head(12)

0     12
1     12
2      1
3      1
4      1
5     12
6      1
7     12
8     12
9     12
10     4
11     4
Name: PeriodFactor, dtype: int64

In [7]:
# n: the number of bonds
n = len(bank_loans)
n

175680

In [8]:
# New column: "Payments": all payment terms
bank_loans['Payments'] = bank_loans['CouponsRemaining'] + bank_loans['CurrentCoupon']

In [9]:
bank_loans['Payments']

0         638
1         670
2          44
3          68
4          56
         ... 
175675    322
175676     93
175677    193
175678    186
175679    367
Name: Payments, Length: 175680, dtype: int64

In [10]:
# New column: 'CouponRatePeriod'
bank_loans['CouponRatePeriod'] = bank_loans['YearlyRate'] / bank_loans['PeriodFactor']

In [11]:
bank_loans['CouponRatePeriod'].head(12)

0     0.011257
1     0.013391
2     0.190721
3     0.092318
4     0.121696
5     0.009176
6     0.151007
7     0.008171
8     0.007386
9     0.008247
10    0.019627
11    0.032176
Name: CouponRatePeriod, dtype: float64

In [12]:
yield_curves.set_index('Date',inplace = True)

In [13]:
yield_curves = yield_curves.drop('Nr',axis =1)

In [14]:
yield_curves.columns = yield_curves.columns.str[:-2]

In [15]:
yield_curves.columns = yield_curves.columns.str[1:]

In [16]:
yield_curves.columns = yield_curves.columns.astype(int) / 100

In [17]:
yield_curves.columns

Float64Index([ 0.25,   0.5,  0.75,   1.0,  1.25,   1.5,  1.75,   2.0,  2.25,
                2.5,
              ...
              27.75,  28.0, 28.25,  28.5, 28.75,  29.0, 29.25,  29.5, 29.75,
               30.0],
             dtype='float64', length=120)

In [18]:
yield_curves.index = pd.to_datetime(yield_curves.index)
yield_curves.index

DatetimeIndex(['1991-01-02', '1991-01-03', '1991-01-04', '1991-01-07',
               '1991-01-08', '1991-01-09', '1991-01-10', '1991-01-11',
               '1991-01-14', '1991-01-15',
               ...
               '2020-08-13', '2020-08-14', '2020-08-17', '2020-08-18',
               '2020-08-19', '2020-08-20', '2020-08-21', '2020-08-24',
               '2020-08-25', '2020-08-26'],
              dtype='datetime64[ns]', name='Date', length=7332, freq=None)

In [19]:
bank_loans['PurchaseDate'] = pd.to_datetime(bank_loans['PurchaseDate'])
bank_loans['PurchaseDate']

0        1994-09-07
1        1994-09-07
2        1994-09-07
3        1994-09-07
4        1994-09-07
            ...    
175675   2005-09-07
175676   2005-09-07
175677   2005-09-07
175678   2005-09-07
175679   2005-09-07
Name: PurchaseDate, Length: 175680, dtype: datetime64[ns]

In [20]:
from scipy.optimize import fsolve
#!pip install numpy_financial
#import numpy_financial as npf

In [21]:
# Function to solve : with multiple rf
def YTM_equation(P0, rf, c, FV, M, s):
    """
    Input: 
    P0: current price (purchase price), 
    rf: a series of rf with M different elements, 
    c: coupon rate for each term, 
    FV: face value 
    M: number of payments
    s: a fixed spread (period)
    """
    # IRR series
    IRR_series = list(map(lambda x:x+s+1, rf)) # IRR = rf + s
    IRR_series.insert(0,1) # 1, rf1+s, rf2+s, ... rfm+s (m+1)
    # Payment sequence
    payment_series = np.repeat(c * FV, M + 1) # Coupon payments, 0,1,...,M
    payment_series[0] = -1 * P0 # At t = 0 we buy the bond.
    payment_series[M] += FV # Last payment includes the principal.

    pv = list()
    for i in range(M):
        pv1 = payment_series[i+1] / (IRR_series[i+1] ** (i+1))
        pv.append(pv1)
        sol = sum(pv) + payment_series[0] # present value of all payments
    return(sol)

In [22]:
# Test for YTM equation
IRR_series = [1, 1.1, 1.2, 1.2]
payment_series = [-80, 5, 5, 105]
M = 3
pv = list()
for i in range(M):
      pv1 = payment_series[i+1] / (IRR_series[i+1] ** (i+1))
      pv.append(pv1)
sol = sum(pv) + payment_series[0] # present value of all payments
sol

-11.218434343434325

In [23]:
def get_rf_series(date, m, periodfactor):
    """
    Input:
    date: date of pricing the bonds
    m: number of future payments
    periodfactor = 12 if monthly paid, 4 if quarterly paid, i if yearly paid
    use m and date as indeces and find rfs from the yield curves
    Return a series of risk free rates
    """
    m_grid = np.arange(1,m+1,1) # m_grid = 1,2,...,m
    factor = np.repeat(1/periodfactor, m)  # 1/12,1/12,...1/12
    m_series = m_grid*factor #1/12, 2/12, ..., m/12
    m_series = np.round(m_series/0.25,0)*0.25 
    m_series = np.where(m_series>30,30,m_series)
    m0 = np.where(m_series==0)[0]
    m_series[m0] = 0.25
    date = np.repeat(date, m)
    rf_yearly = yield_curves.loc[date,m_series]
    rf_yearly.iloc[:,0]=0
    rf_series_one_bond = rf_yearly/periodfactor
    rf_series_one_bond = rf_series_one_bond.iloc[0,:]
    return rf_series_one_bond

In [22]:
###Test
m=12
periodfactor=12
date = bank_loans['PurchaseDate'][0]

m_grid = np.arange(1,m+1,1) # m_grid = 1,2,...,m
factor = np.repeat(1/periodfactor, m)  # 1/12,1/12,...1/12
m_series = m_grid*factor #1/12, 2/12, ..., m/12
m_series = np.round(m_series/0.25,0)*0.25 
m_series = np.where(m_series>30,30,m_series)
#print(m_series)
m0 = np.where(m_series==0)[0]
#print(m0)
m_series[m0] = 0.25
#print(m_series)
date = np.repeat(date, m)
#print(date)
rf_yearly = yield_curves.loc[date,m_series]
#print(rf_yearly)
#print(rf_yearly.iloc[:,0])
rf_yearly.iloc[:,0]=0
rf_series_one_bond = rf_yearly/periodfactor
rf_series_one_bond = rf_series_one_bond.iloc[0,:]
rf_series_one_bond

0.25    0.000000
0.25    0.004709
0.25    0.004709
0.25    0.004709
0.50    0.005187
0.50    0.005187
0.50    0.005187
0.75    0.005578
0.75    0.005578
0.75    0.005578
1.00    0.005883
1.00    0.005883
Name: 1994-09-07 00:00:00, dtype: float64

In [24]:
###Test
P0 = bank_loans['PurchasePrice'][0]
d = bank_loans['PurchaseDate'][0]
f = bank_loans['PeriodFactor'][0]
m = bank_loans['Payments'][0]
FV = bank_loans['FaceValue'][0]
c = bank_loans['CouponRatePeriod'][0]

fsolve(lambda x: YTM_equation(s=x, P0=P0,rf = get_rf_series(date=d, m=m, periodfactor=f),FV = FV,c=c,M=m), x0=0,xtol = 1.49012e-10)

array([0.01080518])

In [25]:
# Compute period spreads 
bank_loans['Spread_Period'] = bank_loans.apply(
    lambda row: fsolve(lambda x: YTM_equation(s=x, P0=row['PurchasePrice'],rf = get_rf_series(date=row['PurchaseDate'], m=row['Payments'], periodfactor=row['PeriodFactor']),FV = row['FaceValue'],c=row['CouponRatePeriod'],M=row['Payments']), x0=0,xtol = 1.49012e-10),
    axis=1)

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [26]:
bank_loans['Spread_Period']

0          [0.010805180252533093]
1          [0.006493492518722265]
2            [0.1693175486071359]
3           [0.04488745089692511]
4           [0.10355520365860739]
                   ...           
175675     [0.004168249313994058]
175676     [0.046082030756982036]
175677     [0.007677713239506383]
175678     [0.007387770040757388]
175679    [0.0022962906408300035]
Name: Spread_Period, Length: 175680, dtype: object

In [27]:
bank_loans['Spread_Period'] = bank_loans['Spread_Period'].apply(lambda x: x[0])

In [29]:
bank_loans['Spread_Period'].head(15)

0     0.010805
1     0.006493
2     0.169318
3     0.044887
4     0.103555
5     0.005607
6     0.134289
7     0.004311
8     0.002665
9     0.004322
10    0.005022
11    0.020145
12    0.085518
13    0.009585
14    0.056523
Name: Spread_Period, dtype: float64

In [30]:
bank_loans['Spread'] = bank_loans['Spread_Period'] * bank_loans['PeriodFactor']

In [31]:
bank_loans['Spread'].head(15)

0     0.129662
1     0.077922
2     0.169318
3     0.044887
4     0.103555
5     0.067282
6     0.134289
7     0.051730
8     0.031983
9     0.051870
10    0.020087
11    0.080581
12    0.085518
13    0.115015
14    0.056523
Name: Spread, dtype: float64

### 1(b) Use the previously calculated **spread**, the **yield curve at September 14th, 2020**, and the **information of the bonds** to price them, so they reflect the current market price for the purposes of capital requirement calculations. Compare the original price with September 15th price. What can you say about the risk-free rates and bond prices?

In [32]:
# Current date
date = yield_curves.index[-1]
date

Timestamp('2020-08-26 00:00:00')

In [33]:
# Solve the current price, using the spreads
bank_loans['CurrentPrice'] = bank_loans.apply(
    lambda row: fsolve(lambda x: YTM_equation(P0=x, rf = get_rf_series(date=date, m=row['CouponsRemaining'], periodfactor=row['PeriodFactor']),FV = row['FaceValue'],c=row['CouponRatePeriod'],M=row['CouponsRemaining'],s=row['Spread_Period']), x0=10000,xtol = 1.49012e-10),
    axis=1)

In [34]:
bank_loans['CurrentPrice']

0         [154767.35622587916]
1          [223936.4625396193]
2         [107027.59766964792]
3         [166108.86186905144]
4          [97638.99089517315]
                  ...         
175675    [110669.39607270088]
175676    [114217.11613815323]
175677     [96983.05807868991]
175678     [98272.03919502818]
175679    [144152.00514958033]
Name: CurrentPrice, Length: 175680, dtype: object

In [35]:
bank_loans['CurrentPrice'] = bank_loans['CurrentPrice'].apply(lambda x: x[0])

In [36]:
bank_loans['CurrentPrice']

0         154767.356226
1         223936.462540
2         107027.597670
3         166108.861869
4          97638.990895
              ...      
175675    110669.396073
175676    114217.116138
175677     96983.058079
175678     98272.039195
175679    144152.005150
Name: CurrentPrice, Length: 175680, dtype: float64

In [37]:
# Price changes
bank_loans['PriceDifference'] = bank_loans['CurrentPrice'] - bank_loans['PurchasePrice']

In [38]:
bank_loans['PriceDifference'] # looks like most are positive

0          56659.626226
1         102631.462540
2          32267.747670
3          94273.571869
4          39723.990895
              ...      
175675     32343.396073
175676     39680.126138
175677     29259.178079
175678     32898.379195
175679     47923.845150
Name: PriceDifference, Length: 175680, dtype: float64

In [45]:
# check those decreased prices: only 1418 bonds have decreased prices
bank_loans[['CurrentPrice','PurchasePrice','PriceDifference']].iloc[np.where(bank_loans['PriceDifference']<0)[0]]

,CurrentPrice,PurchasePrice,PriceDifference
192,169715.649613,173844.39,-4128.740387
1406,80025.103813,80095.89,-70.786187
1411,77872.623053,77995.23,-122.606947
1412,67894.421569,67899.30,-4.878431
1416,79150.543294,79179.54,-28.996706
...,...,...,...
174708,89355.899494,90712.53,-1356.630506
174711,65779.695413,65805.00,-25.304587
174712,68820.954423,69342.66,-521.705577
174713,82923.446695,83922.24,-998.793305


In [46]:
# % Price changes 
bank_loans['PriceChange%'] = ( bank_loans['CurrentPrice'] - bank_loans['PurchasePrice'] ) / bank_loans['PurchasePrice']

In [48]:
# the percentile change is significant
bank_loans['PriceChange%'].mean()

0.42980876990253003

### c. (20%) Calculate the total provisions and capital requirement each operation brings using the prices you just calculated. Report the total provision for the business line and the total capital requirement for that business line. For LGD in corporate, assume the standard Basel weights (i.e. use foundational approach). 

In [49]:
# Time to maturity: when TTM > 30, make it 30
bank_loans['TTM'] = bank_loans['CouponsRemaining'] / bank_loans['PeriodFactor']
bank_loans['TTM_Upper30'] = np.where(bank_loans['TTM']>30,30,bank_loans['TTM'])

In [50]:
bank_loans['TTM_Upper30']

0         27.250000
1         29.916667
2         19.000000
3         30.000000
4         30.000000
            ...    
175675    11.916667
175676    30.000000
175677     1.166667
175678     0.583333
175679    15.666667
Name: TTM_Upper30, Length: 175680, dtype: float64

In [51]:
# percentiles
percentile5 = np.percentile(bank_loans['Spread'], 5)
percentile20 = np.percentile(bank_loans['Spread'], 20)
percentile40 = np.percentile(bank_loans['Spread'], 40)
percentile60 = np.percentile(bank_loans['Spread'], 60)
percentile75 = np.percentile(bank_loans['Spread'], 75)
percentile90 = np.percentile(bank_loans['Spread'], 90)

In [53]:
percentile90

0.162702463741515

In [54]:
# PD
bank_loans['PD'] = 0.001 # lower than 5%
bank_loans['PD'] = np.where((bank_loans['Spread'] > percentile5) & (bank_loans['Spread'] <= percentile20), 0.01, bank_loans['PD'])
bank_loans['PD'] = np.where((bank_loans['Spread'] > percentile20) & (bank_loans['Spread'] <= percentile40), 0.12, bank_loans['PD'])
bank_loans['PD'] = np.where((bank_loans['Spread'] > percentile40) & (bank_loans['Spread'] <= percentile60), 0.36, bank_loans['PD'])
bank_loans['PD'] = np.where((bank_loans['Spread'] > percentile60) & (bank_loans['Spread'] <= percentile75), 0.52, bank_loans['PD'])
bank_loans['PD'] = np.where((bank_loans['Spread'] > percentile75) & (bank_loans['Spread'] <= percentile90), 0.75, bank_loans['PD'])
bank_loans['PD'] = np.where(bank_loans['Spread'] > percentile90, 0.85, bank_loans['PD'])

In [55]:
bank_loans['PD']

0         0.75
1         0.36
2         0.85
3         0.01
4         0.52
          ... 
175675    0.12
175676    0.01
175677    0.36
175678    0.36
175679    0.01
Name: PD, Length: 175680, dtype: float64

In [68]:
# EAD
bank_loans['EAD'] = bank_loans['CurrentPrice']

In [69]:
bank_loans['EAD']

0         154767.356226
1         223936.462540
2         107027.597670
3         166108.861869
4          97638.990895
              ...      
175675    110669.396073
175676    114217.116138
175677     96983.058079
175678     98272.039195
175679    144152.005150
Name: EAD, Length: 175680, dtype: float64

In [70]:
bank_loans['LGD'] = 0.75

In [71]:
bank_loans['LGD']

0         0.75
1         0.75
2         0.75
3         0.75
4         0.75
          ... 
175675    0.75
175676    0.75
175677    0.75
175678    0.75
175679    0.75
Name: LGD, Length: 175680, dtype: float64

In [72]:
bank_loans['Provisions'] = bank_loans['PD'] * bank_loans['LGD'] * bank_loans['EAD'] * bank_loans['BondsHeld']

In [73]:
bank_loans['Provisions']

0         2.263473e+06
1         5.441656e+05
2         6.823009e+05
3         7.350317e+05
4         2.703624e+06
              ...     
175675    1.543838e+06
175676    2.569885e+04
175677    7.855628e+04
175678    3.980018e+05
175679    8.108550e+05
Name: Provisions, Length: 175680, dtype: float64

In [77]:
def total_capital_requirement(PD, LGD, EAD, TTM, N_bonds):
    from scipy.stats import norm
    b = (0.11852-0.05478*np.log(PD))**2
    R =  0.12 * ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) )
    R += 0.24 * (1 - ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) )
    # Time factor 
    time_factor = (1+(TTM-2.5)*b)/(1-1.5*b)
    # Now we can calculate the capital
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD
    K *= time_factor
    K *= EAD 
    K *= N_bonds
    return K 

In [78]:
bank_loans['CapRequirements'] = bank_loans.apply(lambda x: total_capital_requirement(x['PD'], x['LGD'], x['EAD'], x['TTM_Upper30'],x['BondsHeld']),axis=1)

In [79]:
bank_loans['CapRequirements']

0         9.805197e+05
1         1.209136e+06
2         1.435885e+05
3         5.767281e+07
4         3.247320e+06
              ...     
175675    7.159795e+06
175676    2.016410e+06
175677    9.127665e+04
175678    4.538884e+05
175679    3.739762e+07
Name: CapRequirements, Length: 175680, dtype: float64

In [84]:
print("The total capital requirement is $", bank_loans['CapRequirements'].sum() )

The total capital requirement is $ 2425107396670.678


In [85]:
print("The total provisions is $", bank_loans['Provisions'].sum() )

The total provisions is $ 1204999612501.959


In [86]:
!pip install xlwt

In [89]:
bank_loans.to_excel('Dataset_1011.xlsx',sheet_name='bank_loans',header=True)